<a href="https://colab.research.google.com/github/alexander-toschev/ai-tools/blob/main/text/text_theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Лекция: Обработка текстов в Python
Цель — познакомиться с основами обработки текстовых данных, включая очистку, векторизацию и классификацию.


## 📘 Что такое текстовые данные
- Неструктурированные: отзывы, чаты, документы
- Полу-структурированные: HTML, XML, JSON
- Структурированные: датафреймы с колонкой `text`


## 🛠 Библиотеки Python для NLP
- `nltk` — базовая обработка (токенизация, стемминг)
- `spaCy` — быстрый и мощный анализ: POS-теги, сущности
- `re` — регулярные выражения
- `scikit-learn` — векторизация текста (TF-IDF)
- `transformers` — предобученные модели BERT, GPT и др.

## 🔧 Очистка текста
- Приведение к нижнему регистру
- Удаление пунктуации, HTML-тегов
- Удаление стоп-слов
- Лемматизация и/или стемминг

## 📚 Что такое лемматизация?
**Лемматизация** — это процесс приведения слова к его базовой словарной форме.

**Примеры:**
- `running`, `ran`, `runs` → `run`
- `was`, `were` → `be`
- `better` → `good`

Цель — привести все формы одного слова к одной базе, чтобы улучшить анализ.


In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

print(lemmatizer.lemmatize("running"))        # ➜ run
print(lemmatizer.lemmatize("better", pos='a')) # ➜ good
print(lemmatizer.lemmatize("feet"))           # ➜ foot

### 📊 Лемматизация vs Стемминг
- **Стемминг** — обрезает окончания слов, быстрый, но грубый.
- **Лемматизация** — учитывает грамматику и использует словари.

|     | Стемминг | Лемматизация |
|-----|----------|---------------|
| Скорость | Быстрее | Медленнее |
| Точность | Ниже | Выше |
| Основа | Обрезка | Словари + грамматика |


## 🔠 Векторизация текста
- **Bag of Words**: частоты слов
- **TF-IDF**: частота с учетом важности
- **Word2Vec / BERT**: контекстные эмбеддинги


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = ["this is a text", "this is another text"]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.toarray())

## 🧠 Пример: Классификация текстов (20 newsgroups)
- Классы: спорт, медицина, компьютеры и т.д.
- Модель: `TfidfVectorizer` + `Naive Bayes`

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

data = fetch_20newsgroups(subset='train', categories=['sci.med', 'rec.sport.baseball'])
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(data.data, data.target)
print(model.predict(["The player hit a home run"]))

## 📌 Вывод
- Текст нужно очищать и нормализовать
- Лемматизация помогает сократить словарь и повысить качество
- TF-IDF — хороший базовый способ векторизации
- Для сложных задач используют предобученные модели (BERT, GPT и др.)

## 🤖 Что такое BERT
**BERT (Bidirectional Encoder Representations from Transformers)** — это предобученная языковая модель от Google, которая понимает контекст слов в обе стороны — и слева, и справа.

Например, слово `bank` в двух разных предложениях:
- "He went to the **bank** to deposit money" → банк
- "She sat by the **bank** of the river" → берег

**BERT** понимает разницу и создает разные векторные представления.

Он используется для:
- Классификации текста
- Распознавания сущностей
- Вопрос-ответ систем
- Суммаризации

In [ ]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')
print(classifier('I really enjoyed this lesson!'))

## 🔢 Сравнение методов векторизации текста


### 1. Bag of Words (BoW)
Превращает текст в вектор частот слов.
- Не учитывает порядок и контекст
- Прост в реализации

**Пример:**
```python
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
X = vec.fit_transform(["I love NLP", "NLP is fun"])
print(X.toarray())
```

### 2. TF-IDF (Term Frequency-Inverse Document Frequency)
Вектор, учитывающий частоту слова в документе и его редкость в коллекции.
- Снижает вес часто встречающихся слов (например, "the")
- Подходит для классификации

**Пример:**
```python
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
X = vec.fit_transform(["this is a text", "this is another text"])
print(X.toarray())
```

### 3. Word2Vec / GloVe
- Каждое слово представляется как плотный вектор (обычно длины 300)
- Похожее значение → похожий вектор
- Требует предварительного обучения или использования готовых весов

**Пример (gensim Word2Vec):**
```python
from gensim.models import Word2Vec
sentences = [["cat", "sits", "on", "the", "mat"]]
model = Word2Vec(sentences, vector_size=50, min_count=1)
print(model.wv['cat'])
```

### 4. BERT / Sentence-BERT
- Векторизует слова или целые предложения
- Учитывает порядок, контекст, грамматику
- Универсален для любых NLP задач

**Пример:**
```python
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embedding = model.encode("Natural Language Processing is cool")
print(embedding.shape)  # ➜ (384,)
```

### 📊 Сравнение методов
| Метод      | Учитывает порядок? | Контекст? | Тип выхода | Размерность |
|------------|--------------------|-----------|------------|-------------|
| BoW        | ❌                 | ❌        | Документ   | #слов       |
| TF-IDF     | ❌                 | ❌        | Документ   | #слов       |
| Word2Vec   | ❌                 | ✅ частично | Слово      | ~300        |
| BERT       | ✅                 | ✅        | Слово/фраза| 768+        |